In [1]:
from src.core.gamma_client import GammaMarketsClient
from src.execution.order_executor import OrderExecutor
from src.core.clob_client import PolymarketClient

gamma_client = GammaMarketsClient()

# Fetch active events with high liquidity and volume
markets = gamma_client.get_markets(
    closed=False,  # Exclude closed markets
    liquidity_num_min=100000.0,  # Minimum liquidity
    volume_num_min=50000.0,  # Minimum trading volume
    start_date_min="2025-02-10",  # Markets starting after this date
    
)

if markets:
    print(f"Total markets found: {len(markets)}")
    for market in markets:
        # Safely access keys using .get() to avoid KeyError
        question = market.get("question", "N/A")
        event_id = market.get("id", "N/A")
        volume = market.get("volume", "N/A")  # Use .get() to handle missing keys
        active = market.get("active", "N/A")
        closed = market.get("closed", "N/A")
        clobTokenIds = market.get("clobTokenIds")
        orderBook = market.get("enableOrderBook", "N/A")
        print(f"Question: {question}, Event ID: {event_id}, Volume: {volume}, Active: {active}, Enabled Order book: {orderBook}")
        print(f"clobTokenIds  {clobTokenIds},")
else:
    print("No markets found matching the criteria.")

Total markets found: 15
Question: Will Bournemouth finish in 2nd place for the 2024-25 EPL season?, Event ID: 523124, Volume: 211076.443568, Active: True, Enabled Order book: True
clobTokenIds  ["15576313442574913402521764960674801503672915218499248228128007520844494427286", "50542257714266742622884001930480754601213392103803280706827500013923368351392"],
Question: Will Brentford finish in 2nd place for the 2024-25 EPL season?, Event ID: 523125, Volume: 232296.590666, Active: True, Enabled Order book: True
clobTokenIds  ["4477077216676555244043217245689278106787916298196387802721996093357618965741", "8522168098161230802844024089128070699875073770724405654015657935229478751710"],
Question: Will Everton finish in 2nd place for the 2024-25 EPL season?, Event ID: 523129, Volume: 400630.893448, Active: True, Enabled Order book: True
clobTokenIds  ["63111461357480006726724481745334342066468177262121041772463186326458176716028", "596557440720203973436972237862371198080327770529806486624744942

In [2]:
# Question: Bitcoin above $84,000 on March 7?, Event ID: 526067 
yes_token= "26208742573420406262318661661944845485925536996268070025890012912394734994313"

client = PolymarketClient()

# Retrieve order book information
orderbook = client.get_order_book(yes_token)
midpoint = client.get_midpoint_price(yes_token)
best_buy = client.get_price(yes_token, "BUY")
best_sell = client.get_price(yes_token, "SELL")
spread = client.get_spread(yes_token)


print("Order Book:")
print(orderbook)
print("\nMarket Information:")
print(f"Midpoint Price: {midpoint}")
print(f"Best Buy Price: {best_buy}")
print(f"Best Sell Price: {best_sell}")
print(f"Spread: {spread}")

Order Book:
OrderBookSummary(market='0xc4ccf5b4e94d9be31598aef97fd85dc0f0d9d18f6415256bf3504029c3e7c054', asset_id='26208742573420406262318661661944845485925536996268070025890012912394734994313', timestamp='1740941027612', bids=[OrderSummary(price='0.01', size='530033'), OrderSummary(price='0.02', size='10000'), OrderSummary(price='0.03', size='5000'), OrderSummary(price='0.04', size='1500'), OrderSummary(price='0.05', size='1300'), OrderSummary(price='0.06', size='1200'), OrderSummary(price='0.07', size='1100'), OrderSummary(price='0.08', size='1000'), OrderSummary(price='0.09', size='900'), OrderSummary(price='0.1', size='800'), OrderSummary(price='0.11', size='6724'), OrderSummary(price='0.2', size='130'), OrderSummary(price='0.26', size='42'), OrderSummary(price='0.28', size='1188'), OrderSummary(price='0.29', size='199.6'), OrderSummary(price='0.35', size='150'), OrderSummary(price='0.38', size='150.79'), OrderSummary(price='0.4', size='160'), OrderSummary(price='0.48', size='500'

In [3]:
from py_clob_client.order_builder.constants import BUY, SELL


# Instantiate the order executor
executor = OrderExecutor()

# Define a signal for a limit order.
# In a real strategy, this signal would be generated dynamically.
signal = {
    "token_id": yes_token,
    "order_type": "limit",
    "side": BUY,
    "price": 0.03,      
    "quantity": 1.0   
}

In [4]:
order_response = executor.execute_signal(signal)
print("\nLimit Order Response:")
print(order_response)

Placing LIMIT order: BUY 1.0 of 26208742573420406262318661661944845485925536996268070025890012912394734994313 at 0.03
Error placing limit order for 26208742573420406262318661661944845485925536996268070025890012912394734994313: Invalid order inputs

Limit Order Response:
None


In [5]:
# If the order_response contains an order ID, cancel the order.
order_id = order_response.get("order_id") if isinstance(order_response, dict) else None
if order_id:
    print(f"\nOrder placed with ID: {order_id}. Now cancelling...")
    cancel_response = executor.cancel_order(order_id)
    print("Cancel Order Response:")
    print(cancel_response)
else:
    print("\nNo order ID found in response; cannot cancel the order.")


No order ID found in response; cannot cancel the order.
